In [1]:
from pathlib import Path
root_dir = Path('../../streamlined_wireframes/')
import os
## Creating a language chain that can help take a github repo and other documentation and create a language chain that can reference those documents when needed.
from langchain.document_loaders import TextLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter, 
    Language,
)
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv
import openai
load_dotenv()

# Get OpenAI and Pinecone API keys
openai.api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_KEY")
openai.organization = os.getenv("OPENAI_ORG")
pinecone_env = os.getenv("PINECONE_ENV")

# Initialize Pinecone
pinecone.init(api_key=pinecone_api_key, environment=pinecone_env)





c:\Users\sjufa\anaconda3\envs\py39\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
from git import Repo

url = "https://github.com/streamlit/streamlit"
repo = Repo.clone_from(url, to_path=f"./example_data/test_repo1")
branch = repo.head.reference
print(branch.name)


develop


In [3]:
root_dir = "./example_data/test_repo1"


docs = []

python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size = 1000, chunk_overlap = 15
)
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        if file.endswith(".py") and "/.venv/" not in dirpath:
            try:
                loader = TextLoader(os.path.join(dirpath, file), encoding="utf-8")
                docs.extend(loader.load_and_split(text_splitter=python_splitter))
            except Exception as e:
                pass
print(f"{len(docs)}")

4625


In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=15
    )

texts = text_splitter.split_documents(docs)
print(f"{len(texts)}")

4625


In [68]:
# Create the embeddings object
embeddings = OpenAIEmbeddings(
    openai_api_key = openai.api_key,
    openai_organization= openai.organization
)

In [78]:
from langchain.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader(
    file_path="./bakespace_api.pdf",
)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, chunk_overlap = 15
)

In [79]:
pages = pdf_loader.load_and_split(text_splitter=text_splitter)

In [80]:
docs = pages

In [89]:
# Connect to the Pinecone vector store
index = pinecone.GRPCIndex('code-llm')

In [82]:
for i, doc in enumerate(docs):
    doc.metadata["id"] = f'Bakespace_{i}'
    doc.metadata['type'] = 'PDF / XML'
    doc.metadata['repo'] = 'Bakespace API'



In [83]:
texts = []
for doc in docs:
    texts.append(doc.page_content)

In [84]:
vectors=embeddings.embed_documents(texts)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)).


In [85]:
len(vectors), len(vectors[0])

(39, 1536)

In [86]:
# Create a new dataframe from the doc data
import pandas as pd

# Set the columns to "text", "id", "metadata"
df = pd.DataFrame(columns=["values", "id", "metadata"])

# Loop through the docs and set the values for each column
for i, doc in enumerate(docs):
    df = df.append(
        {
            "values": vectors[i],
            "id": doc.metadata["id"],
            "metadata": doc.metadata,
        },
        ignore_index=True,
    )

C:\Users\sjufa\AppData\Local\Temp\ipykernel_26836\464683487.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\sjufa\AppData\Local\Temp\ipykernel_26836\464683487.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\sjufa\AppData\Local\Temp\ipykernel_26836\464683487.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\sjufa\AppData\Local\Temp\ipykernel_26836\464683487.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\sjufa\AppData\Local\Temp\ipykernel_26836\464683487.py:9: FutureWarning: The frame.append method is deprecated and will be r

In [87]:
df.head()

,values,id,metadata
0,"[0.013707088936454722, 0.013427779695556452, 0...",Bakespace_0,"{'source': './bakespace_api.pdf', 'page': 0, '..."
1,"[-0.011386240434354072, 0.023548186928008977, ...",Bakespace_1,"{'source': './bakespace_api.pdf', 'page': 0, '..."
2,"[0.0006587831557177874, 0.0036708665097053145,...",Bakespace_2,"{'source': './bakespace_api.pdf', 'page': 1, '..."
3,"[0.002096409684365405, -0.010518363715944229, ...",Bakespace_3,"{'source': './bakespace_api.pdf', 'page': 1, '..."
4,"[0.007804707419193777, -0.005885352577264909, ...",Bakespace_4,"{'source': './bakespace_api.pdf', 'page': 1, '..."


In [90]:
index.upsert_from_dataframe(df, batch_size=50)

sending upsert requests:   0%|          | 0/39 [00:00<?, ?it/s]

collecting async responses:   0%|          | 0/1 [00:00<?, ?it/s]

upserted_count: 39

In [91]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.3,
 'namespaces': {'': {'vector_count': 27173}},
 'total_vector_count': 27173}

In [95]:
vectorstore = Pinecone.from_existing_index(index_name="code-llm", embedding = embeddings)

In [97]:
query = "what does the generate_cocktail function do?"

vectorstore.similarity_search(query, k=5)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)).
Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.


[]